In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from encode_data import *
from midi_data import *

In [4]:
from tqdm import tqdm
import pandas as pd
from data_sources import process_parallel, transform_csv_row

In [5]:
from collections import Counter

In [6]:
import scipy.sparse

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [7]:
version = 'v9'
data_path = Path('data/midi')
version_path = data_path/version

In [8]:
source_dir = 'midi_transform'
out_dir = 'midi_npz'
source_csv = version_path/source_dir/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'{out_dir}.csv'

In [9]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,4,15,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,ht_bpm,artist,ht_key,ht_time_signature,bpm,section,parts,genres,mxl,midi_transform
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1.0,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,...,85.0,wayne-sharpe,C,3.0,85.0,intro,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,128.0,wayne-sharpe,C,4.0,128.0,chorus,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,96.0,what-a-day,C,4.0,96.0,chorus,chorus,Jazz,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1.0,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,128.0,weezer,C,4.0,NaN,intro-and-verse,intro-and-verse,NaN,NaN,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,93.0,weezer,C,4.0,93.0,chorus,chorus,Rock,NaN,midi_transform/hooktheory/pianoroll/w/weezer/f...


In [10]:
df_filtered = df.loc[df[source_dir].notna()]; df_filtered.shape

(163266, 23)

In [11]:
def transform_func(file, out_file, row):
    stream = file2stream(file) # 1.
    chordarr = stream2chordarr(stream, max_dur=128) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
    if row.source != 'hooktheory': # keep hooktheory
        chord_short = compress_chordarr(chordarr)
        delta_trim = chordarr.shape[0] - chord_short.shape[0]
        if delta_trim > 100: print(f'Removed {delta_trim} rests from {file}')
        chordarr = chord_short
    save_chordarr(out_file, chordarr)

In [12]:
from functools import partial
parallel_func = partial(transform_csv_row, 
        transform_func=transform_func,
        base_path=version_path,
        source_dir=source_dir,
        out_dir=out_dir,
        out_extension='.npz'
       )

In [13]:
# for r in df_filtered.iterrows():
#     parallel_func(r)
#     break

In [14]:
transcribed_files = process_parallel(parallel_func, df_filtered.iterrows(), total=df_filtered.shape[0])

Compressing rests: 56 -> 32
Compressing rests: 128 -> 32
Removed 128 rests from data/midi/v9/midi_transform/freemidi/genre-dance-eletric/Eurythmics - Revival.mid
Compressing rests: 48 -> 32
Compressing rests: 192 -> 32
Removed 176 rests from data/midi/v9/midi_transform/freemidi/genre-dance-eletric/Madonna - Beautiful Stranger.mid
Compressing rests: 64 -> 32
Removed 224 rests from data/midi/v9/midi_transform/freemidi/genre-dance-eletric/Madonna - Frozen.mid
Compressing rests: 47 -> 35
Compressing rests: 55 -> 35
Compressing rests: 64 -> 32
Compressing rests: 52 -> 32
Compressing rests: 58 -> 34
Compressing rests: 42 -> 34
Compressing rests: 64 -> 32
Removed 132 rests from data/midi/v9/midi_transform/freemidi/genre-dance-eletric/Fatboy Slim - Praise You.mid
Compressing rests: 128 -> 32
Compressing rests: 42 -> 34
Compressing rests: 44 -> 32
Compressing rests: 124 -> 32
Error converting midi to sequence index 127 is out of bounds for axis 2 with size 127 data/midi/v9/midi_transform/freemi

Compressing rests: 102 -> 34
Compressing rests: 97 -> 33
Compressing rests: 112 -> 32
Compressing rests: 98 -> 34
Compressing rests: 98 -> 34
Compressing rests: 99 -> 35
Compressing rests: 116 -> 32
Compressing rests: 111 -> 35
Compressing rests: 102 -> 34
Compressing rests: 102 -> 34
Compressing rests: 91 -> 35
Compressing rests: 120 -> 32
Compressing rests: 90 -> 34
Compressing rests: 96 -> 32
Compressing rests: 111 -> 35
Compressing rests: 86 -> 34
Compressing rests: 126 -> 34
Compressing rests: 86 -> 34
Compressing rests: 90 -> 34
Compressing rests: 76 -> 32
Compressing rests: 125 -> 33
Compressing rests: 80 -> 32
Compressing rests: 128 -> 32
Compressing rests: 85 -> 33
Compressing rests: 74 -> 34
Compressing rests: 130 -> 34
Compressing rests: 83 -> 35
Compressing rests: 82 -> 34
Compressing rests: 72 -> 32
Compressing rests: 134 -> 34
Compressing rests: 126 -> 34
Compressing rests: 86 -> 34
Compressing rests: 87 -> 35
Compressing rests: 46 -> 34
Compressing rests: 138 -> 34
Compr

Compressing rests: 194 -> 34
Compressing rests: 195 -> 35
Compressing rests: 195 -> 35
Compressing rests: 197 -> 33
Compressing rests: 316 -> 32
Compressing rests: 189 -> 33
Compressing rests: 189 -> 33
Compressing rests: 189 -> 33
Compressing rests: 191 -> 35
Compressing rests: 311 -> 35
Compressing rests: 183 -> 35
Compressing rests: 183 -> 35
Compressing rests: 183 -> 35
Compressing rests: 185 -> 33
Compressing rests: 305 -> 33
Compressing rests: 177 -> 33
Compressing rests: 177 -> 33
Compressing rests: 177 -> 33
Compressing rests: 179 -> 35
Compressing rests: 299 -> 35
Compressing rests: 40 -> 32
Compressing rests: 171 -> 35
Compressing rests: 38 -> 34
Compressing rests: 213 -> 33
Compressing rests: 211 -> 35
Compressing rests: 217 -> 33
Compressing rests: 341 -> 33
Compressing rests: 211 -> 35
Compressing rests: 213 -> 33
Compressing rests: 211 -> 35
Compressing rests: 217 -> 33
Compressing rests: 341 -> 33
Compressing rests: 211 -> 35
Compressing rests: 213 -> 33
Compressing rest

Compressing rests: 64 -> 32
Compressing rests: 64 -> 32
Compressing rests: 81 -> 33
Compressing rests: 64 -> 32
Removed 208 rests from data/midi/v9/midi_transform/freemidi/genre-dance-eletric/New Order - Confusion.mid
Error converting midi to sequence cannot reshape array of size 0 into shape (0,newaxis) data/midi/v9/midi_transform/freemidi/genre-dance-eletric/Daft Punk - Aerodynamic.mid
Compressing rests: 113 -> 33
Compressing rests: 58 -> 34
Compressing rests: 60 -> 32
Compressing rests: 128 -> 32
Removed 124 rests from data/midi/v9/midi_transform/freemidi/genre-dance-eletric/Breathe Carolina - No Vacancy.mid
Compressing rests: 248 -> 32
Compressing rests: 252 -> 32
Removed 548 rests from data/midi/v9/midi_transform/freemidi/genre-dance-eletric/Fatboy Slim - Rockafella Skank.mid
Compressing rests: 58 -> 34
Compressing rests: 47 -> 35
Compressing rests: 111 -> 35
Compressing rests: 63 -> 35
Removed 108 rests from data/midi/v9/midi_transform/freemidi/genre-dance-eletric/Vengaboys - Boo

Removed 256 rests from data/midi/v9/midi_transform/freemidi/genre-pop/Box Car Racer - There Is (2).mid
Error converting midi to sequence list index out of range data/midi/v9/midi_transform/freemidi/genre-pop/Rihanna - Disturbia.mid
Removed 1712 rests from data/midi/v9/midi_transform/freemidi/genre-pop/Backstreet Boys - Spanish Eyes.mid
Removed 304 rests from data/midi/v9/midi_transform/freemidi/genre-pop/Anita Harris - Playground.mid
Compressing rests: 92 -> 32
Removed 108 rests from data/midi/v9/midi_transform/freemidi/genre-pop/David Bowie - Dancing In The Street.mid
Compressing rests: 192 -> 32
Removed 176 rests from data/midi/v9/midi_transform/freemidi/genre-pop/Madonna - Beautiful Stranger.mid
Removed 112 rests from data/midi/v9/midi_transform/freemidi/genre-pop/Scorpions - Cant Get Enough.mid
Compressing rests: 83 -> 35
Removed 112 rests from data/midi/v9/midi_transform/freemidi/genre-pop/Britney Spears - till the world ends.mid
Compressing rests: 288 -> 32
Removed 256 rests from

...

Removed 128 rests from data/midi/v9/midi_transform/130k_reddit/T/T/THE BEATLES.Bogey music K.mid
Compressing rests: 68 -> 32
Compressing rests: 52 -> 32
Removed 120 rests from data/midi/v9/midi_transform/130k_reddit/T/T/TV_Themes_-_Hogan's_Heroes.mid
Compressing rests: 103 -> 35
Compressing rests: 112 -> 32
Removed 244 rests from data/midi/v9/midi_transform/130k_reddit/T/T/time-8.mid
Error converting midi to sequence index 127 is out of bounds for axis 2 with size 127 data/midi/v9/midi_transform/130k_reddit/T/T/thacrossroads.mid
Compressing rests: 346 -> 34
Removed 312 rests from data/midi/v9/midi_transform/130k_reddit/T/T/take_me_there.mid
Compressing rests: 42 -> 34
Compressing rests: 38 -> 34
Compressing rests: 52 -> 32
Compressing rests: 48 -> 32
Compressing rests: 143 -> 35
Removed 188 rests from data/midi/v9/midi_transform/130k_reddit/T/T/time-9.mid
Compressing rests: 128 -> 32
Compressing rests: 92 -> 32
Removed 156 rests from data/midi/v9/midi_transform/130k_reddit/T/T/The-Prom

Removed 544 rests from data/midi/v9/midi_transform/130k_reddit/T/T/The-Lady-In-My-Life-1.mid
Compressing rests: 64 -> 32
Compressing rests: 46 -> 34
Compressing rests: 50 -> 34
Compressing rests: 46 -> 34
Compressing rests: 45 -> 33
Removed 116 rests from data/midi/v9/midi_transform/130k_reddit/T/T/tcha3_3.mid
Compressing rests: 48 -> 32
Removed 112 rests from data/midi/v9/midi_transform/130k_reddit/T/T/Terpurukku_Disini.mid
Compressing rests: 128 -> 32
Removed 144 rests from data/midi/v9/midi_transform/130k_reddit/T/T/This-Must-Be-Love.mid
Removed 144 rests from data/midi/v9/midi_transform/130k_reddit/T/T/THE CURE.Close to me K.mid
Compressing rests: 313 -> 33
Compressing rests: 63 -> 35
Compressing rests: 63 -> 35
Removed 512 rests from data/midi/v9/midi_transform/130k_reddit/T/T/taintedloveexmx.mid
Compressing rests: 44 -> 32
Removed 160 rests from data/midi/v9/midi_transform/130k_reddit/T/T/The-Spirit.mid
Removed 160 rests from data/midi/v9/midi_transform/130k_reddit/T/T/Tenth man 

Compressing rests: 58 -> 34
Compressing rests: 134 -> 34
Compressing rests: 92 -> 32
Compressing rests: 64 -> 32
Compressing rests: 64 -> 32
Compressing rests: 56 -> 32
Compressing rests: 41 -> 33
Compressing rests: 48 -> 32
Compressing rests: 38 -> 34
Compressing rests: 110 -> 34
Compressing rests: 56 -> 32
Compressing rests: 76 -> 32
Compressing rests: 44 -> 32
Compressing rests: 62 -> 34
Compressing rests: 63 -> 35
Compressing rests: 63 -> 35
Removed 164 rests from data/midi/v9/midi_transform/130k_reddit/2/2BRICH.MID
Compressing rests: 89 -> 33
Removed 17600 rests from data/midi/v9/midi_transform/130k_reddit/T/T/take503.mid
Compressing rests: 106 -> 34
Compressing rests: 54 -> 34
Compressing rests: 107 -> 35
Compressing rests: 159 -> 35
Compressing rests: 106 -> 34
Removed 392 rests from data/midi/v9/midi_transform/130k_reddit/2/2_Unlimited_-_No_Limits.mid
Compressing rests: 128 -> 32
Compressing rests: 128 -> 32
Compressing rests: 192 -> 32
Removed 352 rests from data/midi/v9/midi_

Compressing rests: 48 -> 32
Compressing rests: 68 -> 32
Compressing rests: 122 -> 34
Removed 104 rests from data/midi/v9/midi_transform/130k_reddit/R/R/right_now.mid
Removed 144 rests from data/midi/v9/midi_transform/130k_reddit/R/R/RedDwarf3.mid
Compressing rests: 61 -> 33
Compressing rests: 304 -> 32
Removed 288 rests from data/midi/v9/midi_transform/130k_reddit/R/R/Renacim.mid
Error converting midi to sequence index 1559 is out of bounds for axis 0 with size 1559 data/midi/v9/midi_transform/130k_reddit/R/R/rockclock.mid
Removed 144 rests from data/midi/v9/midi_transform/130k_reddit/R/R/Rolling_Stones_-_Paint_It_Black.mid
Compressing rests: 96 -> 32
Error converting midi to sequence  data/midi/v9/midi_transform/130k_reddit/R/R/Rob_Zombie_-_Demonoid_Phenomenon.mid
Compressing rests: 96 -> 32
Compressing rests: 58 -> 34
Removed 896 rests from data/midi/v9/midi_transform/130k_reddit/R/R/Renato_e_seu__Blue_Caps__Meu_bem_nao_me_quer.mid
Compressing rests: 64 -> 32
Removed 288 rests from d

Removed 112 rests from data/midi/v9/midi_transform/130k_reddit/R/R/Radiohead_-_Climbing_Up_the_Walls.mid
Removed 1760 rests from data/midi/v9/midi_transform/130k_reddit/R/R/Robson_Monteiro__Eh_preciso_aceitar.mid
Error converting midi to sequence index 1559 is out of bounds for axis 0 with size 1559 data/midi/v9/midi_transform/130k_reddit/R/R/ROCKCLOC.MID
Removed 192 rests from data/midi/v9/midi_transform/130k_reddit/R/R/Red-Sun-(Modern-Style).mid
Compressing rests: 61 -> 33
Removed 108 rests from data/midi/v9/midi_transform/130k_reddit/R/R/radioa.mid
Removed 2016 rests from data/midi/v9/midi_transform/130k_reddit/R/R/revol123.mid
Compressing rests: 84 -> 32
Compressing rests: 42 -> 34
Compressing rests: 55 -> 35
Compressing rests: 128 -> 32
Removed 144 rests from data/midi/v9/midi_transform/130k_reddit/R/R/roses_are_red.mid
Compressing rests: 71 -> 35
Error converting midi to sequence index 127 is out of bounds for axis 2 with size 127 data/midi/v9/midi_transform/130k_reddit/R/R/Roxan

In [15]:
tdf = pd.DataFrame(data={out_dir: list(transcribed_files.values())}, index=list(transcribed_files.keys()))

In [16]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

((163266, 1), (185846, 23), (185846, 24))

In [17]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,artist,ht_key,ht_time_signature,bpm,section,parts,genres,mxl,midi_transform,midi_npz
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1.0,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,...,wayne-sharpe,C,3.0,85.0,intro,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,wayne-sharpe,C,4.0,128.0,chorus,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,what-a-day,C,4.0,96.0,chorus,chorus,Jazz,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1.0,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,weezer,C,4.0,NaN,intro-and-verse,intro-and-verse,NaN,NaN,NaN,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,weezer,C,4.0,93.0,chorus,chorus,Rock,NaN,midi_transform/hooktheory/pianoroll/w/weezer/f...,midi_npz/hooktheory/pianoroll/w/weezer/fall-to...


### Calculate timesteps

In [18]:
merged_df = pd.read_csv(out_csv)

In [19]:
def calc_timesteps(idxrow):
    idx,row = idxrow
    np_file = row[out_dir]
    if not isinstance(np_file, str) or not (Path(version_path)/np_file).exists(): return idx,None
    try:
#         timesteps = scipy.sparse.load_npz(np_file).shape[0]
        timesteps = load_chordarr(Path(version_path)/np_file).shape[0]
        return idx,timesteps
    except Exception as e:
        print('Error reading text', e, np_file)
    return idx, None

In [20]:
# for r in merged_df.iterrows():
#     calc_timesteps(r)

In [21]:
file2steps = process_parallel(calc_timesteps, merged_df.iterrows(), total=merged_df.shape[0])

In [22]:
len_df = pd.DataFrame(data={f'{out_dir}_timesteps': list(file2steps.values())}, index=list(file2steps.keys()))
merged_len_df = merged_df.join(len_df, how='outer');
len_df.shape, merged_df.shape, merged_len_df.shape

((185846, 1), (185846, 24), (185846, 25))

In [23]:
merged_len_df.to_csv(out_csv, index=False); merged_len_df.head()

,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,ht_key,ht_time_signature,bpm,section,parts,genres,mxl,midi_transform,midi_npz,midi_npz_timesteps
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1.0,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,...,C,3.0,85.0,intro,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,C,4.0,128.0,chorus,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,C,4.0,96.0,chorus,chorus,Jazz,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1.0,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,C,4.0,NaN,intro-and-verse,intro-and-verse,NaN,NaN,NaN,NaN,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,C,4.0,93.0,chorus,chorus,Rock,NaN,midi_transform/hooktheory/pianoroll/w/weezer/f...,midi_npz/hooktheory/pianoroll/w/weezer/fall-to...,65.0
